In [2]:
from genepy.utils import helper as h

from taigapy import TaigaClient
tc = TaigaClient()


In [1]:
from depmapomics import tracker
track = tracker.getTracker()

In [4]:
track[(track.low_quality==1)&(track.datatype=='wgs')]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_line_name,participant_id,cellosaurus_id,...,19q3,19q4,20q1,20q2,20q4,20q,21Q1,21Q2,parent_id,21Q3
CDS-dIijHP,ACH-000309,1,NaN,NaN,wgs,7.169431e+10,NaN,SKLU1,PT-FmbFY70A,CVCL_0629,...,1.0,1.0,1.0,1.0,NaN,NaN,0.0,1.0,NaN,NaN
CDS-ovDo8V,ACH-000890,1,NaN,NaN,wgs,9.124674e+10,NaN,SW1271,PT-a7rG9fcn,CVCL_1716,...,1.0,1.0,1.0,1.0,NaN,NaN,0.0,1.0,NaN,NaN
CDS-mYOC4j,ACH-000893,1,NaN,NaN,wgs,1.432370e+11,NaN,NCIH1651,PT-LRgbHk7Q,CVCL_1484,...,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN
CDS-Wszh2o,ACH-000860,1,NaN,NaN,wgs,8.011841e+10,NaN,NCIH358,PT-HLoL0NZn,CVCL_1559,...,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN
CDS-3PGQ84,ACH-001553,1,NaN,NaN,wgs,7.667525e+10,NaN,MEGA2,PT-SMrpbPt7,CVCL_1832,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
CDS-AqZLna,ACH-002512,1,NaN,NaN,wgs,8.103403e+10,NaN,M1610113,PT-pUBtKkYR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN


In [34]:
import numpy as np
from contrastive import CPCA

N = 400
D = 30
gap = 3
# In B, all the data pts are from the same distribution, which has different variances in three subspaces.
B = np.zeros((N-100, D))
B[:, 0:10] = np.random.normal(0, 10, (N-100, 10))
B[:, 10:20] = np.random.normal(0, 3, (N-100, 10))
B[:, 20:30] = np.random.normal(0, 1, (N-100, 10))


# In A there are four clusters.
A = np.zeros((N, D))
A[:, 0:10] = np.random.normal(0, 10, (N, 10))
# group 1
A[0:100, 10:20] = np.random.normal(0, 1, (100, 10))
A[0:100, 20:30] = np.random.normal(0, 1, (100, 10))
# group 2
A[100:200, 10:20] = np.random.normal(0, 1, (100, 10))
A[100:200, 20:30] = np.random.normal(gap, 1, (100, 10))
# group 3
A[200:300, 10:20] = np.random.normal(2*gap, 1, (100, 10))
A[200:300, 20:30] = np.random.normal(0, 1, (100, 10))
# group 4
A[300:400, 10:20] = np.random.normal(2*gap, 1, (100, 10))
A[300:400, 20:30] = np.random.normal(gap, 1, (100, 10))
A_labels = [0]*100+[1]*100+[2]*100+[3]*100

cpca = CPCA(standardize=False, n_components=10)
proj = cpca.fit_transform(A, B, active_labels=A_labels)


In [36]:
proj[0][0,3]

-18.02944285434579

In [2]:
from genepy.utils import helper as h

from taigapy import TaigaClient
tc = TaigaClient()

my_alligner = Celligner()

# load from taiga public (figshare link)
# load internal expression,
CCLE_expression = tc.get(name='internal-21q3-fe4c', version=15,
                         file='CCLE_expression_full')  # https://depmap.org/portal/download/

# load  TCGA expression
# this dataset was generated from  ,using this script: 
TCGA_expression = tc.get(name='celligner-input-9827',
                         version=1, file='tumor_expression')

# subset gene names to ensembl ids only
TCGA_expression =

CCLE_expression = CCLE_expression[CCLE_expression.columns[:-92]]
CCLE_expression.columns = list(map(lambda x: x.split(
    ' (')[1][:-1] if ' (' in x else x, CCLE_expression.columns))

my_alligner.fit(TCGA_expression)

my_alligner.predict(CCLE_expression)



In [3]:
genename = h.generateGeneNames(ensemble_server="http://nov2020.archive.ensembl.org/biomart",
                    useCache=True, cache_folder=".genenames/", attributes=[])

# drops duplicates keeping the first occurrence


fetching gene names from biomart cache


In [7]:
a = list(set(genename['gene_biotype']))

In [53]:
hgnc_complete_set = tc.get(
    name='hgnc-87ab', version=5, file='hgnc_complete_set')

ACH = tc.get(name='arxspan-cell-line-export-f808', version=752, file='ACH')

CCLE_annotations = tc.get(name='celligner-input-9827',
                          version=1, file='CCLE_annotations')

tumor_annotations = tc.get(name='celligner-input-9827',
                           version=1, file='tumor_annotations')
                           
tumor_expression = tc.get(name='celligner-input-9827',
                          version=1, file='tumor_expression')


In [51]:
set(CCLE_expression.columns) & set(['ENSG00000004866'])


{'ENSG00000004866'}